In [ ]:
# This is needed to fix the sqlite3 import issue
# you might not needed it
__import__('pysqlite3')
import sys
import os
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join('.', 'db.sqlite3'),
    }
}

In [ ]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import os
from git import Repo

In [ ]:
repo_path = "./test_repo"
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [5]:
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/core/langchain_core/",
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf-8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents = loader.load()
len(documents)

488

In [6]:
python_spliter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap = 200
)

texts = python_spliter.split_documents(documents)
len(texts)

1473

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k": 8},
)

In [9]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=200,
)

In [10]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Você é um revisor de código experiente. Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseadas no contexto fornecido abaixo: \n\n{context}",
    ),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [11]:
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para o código de RunnableBinding"})
response["answer"]

'1. No método `bind`, a documentação poderia ser mais informativa. Por exemplo, fornecer uma explicação de como os `kwargs` são utilizados e talvez um exemplo de uso.\n2. No método `with_config`, a tipagem de `cast("RunnableConfig", {**self.config, **(config or {}), **kwargs})` pode ser desnecessária, pois o objeto `config` é do tipo `RunnableConfig` opcional.\n3. No método `_get_runnable_config_param`, a variável `type_hints` poderia ser renomeada para algo mais descritivo, como `annotations`.\n4. No método `_on_run_update`, seria útil adicionar mais comentários para explicar a lógica do código, como a verificação de erro e a chamada de funções com argumentos variáveis.\n5. No método `_executor`, seria interessante adicionar um comentário explicando o motivo de usar'

In [12]:
print(response["answer"])

1. No método `bind`, a documentação poderia ser mais informativa. Por exemplo, fornecer uma explicação de como os `kwargs` são utilizados e talvez um exemplo de uso.
2. No método `with_config`, a tipagem de `cast("RunnableConfig", {**self.config, **(config or {}), **kwargs})` pode ser desnecessária, pois o objeto `config` é do tipo `RunnableConfig` opcional.
3. No método `_get_runnable_config_param`, a variável `type_hints` poderia ser renomeada para algo mais descritivo, como `annotations`.
4. No método `_on_run_update`, seria útil adicionar mais comentários para explicar a lógica do código, como a verificação de erro e a chamada de funções com argumentos variáveis.
5. No método `_executor`, seria interessante adicionar um comentário explicando o motivo de usar
